In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def get_products(name_product, minimal_price_limit, maximal_price_limit):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

    products = []
    for repeticion in range(5):
        rango = 0
        if repeticion > 0:
            rango = str(repeticion*50+1)
            response = requests.get('https://listado.mercadolibre.com.mx/'+name_product.replace(' ', '-')+'_Desde_'+rango+'_OrderId_PRICE_PriceRange_'+minimal_price_limit+'-'+maximal_price_limit+'_ITEM*CONDITION_2230284_NoIndex_True_SHIPPING*ORIGIN_10215068', headers=headers)
        else:
            response = requests.get('https://listado.mercadolibre.com.mx/'+name_product.replace(' ', '-')+'_OrderId_PRICE_PriceRange_'+minimal_price_limit+'-'+maximal_price_limit+'_ITEM*CONDITION_2230284_NoIndex_True_SHIPPING*ORIGIN_10215068', headers=headers)
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        for item in soup.select('.poly-card'):
            title_tag = item.select_one('.poly-component__title-wrapper')
            price_tag = item.select_one('.andes-money-amount--cents-superscript').select_one('.andes-money-amount__fraction')
            link_tag = item.select_one('.poly-component__title')
            if not title_tag or not price_tag or not link_tag:
                continue

            title = title_tag.text.strip()
            price = price_tag.text.strip()
            link = link_tag['href']
            #products.append({'title': title, 'price': price, 'link': link})
            products.append({'title': title, 'price': price, 'link': link})
        #print("Products:", products)
    return products


In [20]:
# Definir variables
product_name = 'silla gamer'
minimal_price_limit = '800'
maximal_price_limit = '3500'

In [21]:
# Obtener productos
products = get_products(product_name, minimal_price_limit, maximal_price_limit)
print("Productos encontrados:", products)

Productos encontrados: [{'title': 'Silla De Escritorio Ergonomica Color Negro', 'price': '800', 'link': 'https://www.mercadolibre.com.mx/silla-de-escritorio-ergonomica-color-negro/p/MLM26042280?pdp_filters=SHIPPING_ORIGIN%3A10215068%7Cprice%3A800-3500&highlight=false&headerTopBrand=false#polycard_client=search-nordic&searchVariation=MLM26042280&position=36&search_layout=grid&type=product&tracking_id=4727dbfc-3bfa-45e3-af17-4861063b1c64&wid=&sid=search'}, {'title': 'Converticama Infantil De Espuma De Poliuretano Niño/niña', 'price': '800', 'link': 'https://articulo.mercadolibre.com.mx/MLM-945134056-converticama-infantil-de-espuma-de-poliuretano-ninonina-_JM?searchVariation=186864627257&highlight=false&pdp_filters=SHIPPING_ORIGIN%3A10215068%7Cprice%3A800-3500&headerTopBrand=false#polycard_client=search-nordic&searchVariation=186864627257&position=14&search_layout=grid&type=item&tracking_id=4727dbfc-3bfa-45e3-af17-4861063b1c64'}, {'title': 'Silla De Escritorio Carnation Negra Con Tapizado

In [22]:
# Convertir a DataFrame
products_df = pd.DataFrame(products)

In [23]:
# Guardar en archivos CSV
products_df.to_csv('productos.csv', index=False)

In [24]:
print("Productos encontrados:", products_df)

Productos encontrados:                                                  title  price  \
0           Silla De Escritorio Ergonomica Color Negro    800   
1    Converticama Infantil De Espuma De Poliuretano...    800   
2    Silla De Escritorio Carnation Negra Con Tapiza...    835   
3    Silla Escritorio Ergonómica Para Niños Color T...    839   
4    Toncoo Portatarjetas De Madera, 10 Portanúmero...    846   
..                                                 ...    ...   
235  Silla Gamer Stylos Stgame1 Ergonomica Reforzad...  1,646   
236  Silla Ergonómica Descanso De Cuello Espalda Ro...  1,646   
237  Silla De Oficina Valery Gris Con Negro Materia...  1,649   
238  Silla Ergonómica de Oficina Reclinable con Rue...  1,649   
239  Silla Gamer Nasa Escritorio Ergonómica Ajustab...  1,659   

                                                  link  
0    https://www.mercadolibre.com.mx/silla-de-escri...  
1    https://articulo.mercadolibre.com.mx/MLM-94513...  
2    https://www.mercadol

In [25]:
# Cargar el archivo CSV
df = pd.read_csv('productos.csv')

In [ ]:

# Lista de palabras clave válidas al inicio del título
palabras_clave_inicio = ['silla', 'gamer', 'ejecutiva']
palabras_prohibidas = ['playa','escritorio', 'bebé', 'bebe', 'mascota', 'auto', 'carro', 'niño', 'niña', 'comedor', 'bar', 'jardín', 'jardin']
def titulo_valido(titulo):
    titulo = str(titulo).lower().strip()
    for palabra in palabras_clave_inicio:
        if titulo.startswith(palabra):
            if any(p in titulo for p in palabras_prohibidas):
                return False
            return True
    return False

In [ ]:
# Aplicar el filtro
df_filtrado = df[df['title'].apply(titulo_valido)]
df_filtrado.to_csv('productos_filtrados_bien.csv', index=False)
